In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os 
from scipy import stats
from pypet import Environment, Trajectory, cartesian_product
import helper_functions as helper
%matplotlib inline

with open('data_path.txt', 'r') as f:
    data_path = os.path.join(f.readline(), 'steering')

hdf_path = os.path.join(data_path, 'HDF')
    
steering_data_path = os.path.join(data_path, 'MarkkulaEtAl_BiolCyb2018_DataWithIntermittentInterpretation.mat')
steering_data = helper.loadmat(steering_data_path)

sightpoint_filepath = os.path.join(data_path, 'PreCalculatedSightPointSignalData.mat')
sightpoint_data = helper.loadmat(sightpoint_filepath)['SSightPointPreCalc']

lk_data = steering_data['STasks'][0]
lk_sightpoint_data = sightpoint_data.STasks[0]

passive_phases_lk = pd.read_csv(os.path.join(data_path, 'passive_phases_lk.csv'), index_col=None)

In [2]:
def interp_p(t_exp, p_exp, vectorized=False):
    # here we take the last 30% of each trial and fit a linear function 
    # to p dynamics to be able to extrapolate beyond observed action point
    # caution: this function is not vectorized because it's going to be used in step-by-step numerical simulations
    # so plotting won't work
    lower_bound = max(int(len(t_exp)*0.3), 2)
    slope, intercept, _, _, _ = sp.stats.linregress(x=t_exp[-lower_bound:], y=p_exp[-lower_bound:])
    
    if not vectorized:
        return lambda t: np.interp(t, t_exp, p_exp) if t<t_exp.max() else intercept + t*slope
    else:
        return lambda t: np.concatenate([np.interp(t[t<t_exp.max()], t_exp, p_exp), 
                                     intercept + t[t>=t_exp.max()]*slope])

In [3]:
p_exp = p_dynamics_lk.loc[1,1,1]
t_exp = np.linspace(0, len(p_exp)/60, len(p_exp))
t_extra = np.linspace(len(p_exp)/60, 2*len(p_exp)/60, len(p_exp))

p_extra = interp_p(t_exp, p_exp, vectorized=True)

plt.plot(t_exp, p_exp)
plt.plot(t_extra, p_extra(t_extra))

NameError: name 'p_dynamics_lk' is not defined